# Trial 3

In [82]:
!pip install autogluon.features --no-deps
!pip install toolz
!pip install gluonts --no-deps
!pip install mlforecast
!pip install autogluon.tabular[lightgbm]==1.1.1


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached scipy-1.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached scikit_learn-1.4.0-1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached lightgbm-4.3.0-py3-none-manylinux_2_28_x86_64.whl.metadata (19 kB)
  Using cached boto3-1.35.49-py3-none-any.whl.metadata (6.7 kB)
  Using cached psutil-5.9.8-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
  Using cached botocore-1.35.49-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-no

In [7]:
import pandas as pd
import numpy as np  
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

In [66]:
train_df = pd.read_csv("/home/yoon/Desktop/Others/open/train.csv")
#test_df = pd.read_csv("")
train_df = train_df[['ID', "timestamp", "supply(kg)", "price(원/kg)"]]
train_df

,ID,timestamp,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,0.0,0.0
1,TG_A_J_20190102,2019-01-02,0.0,0.0
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0
...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,477220.0,523.0


In [127]:
weather_2019 = pd.read_csv("weather_average2019.csv")
weather_2019.rename(columns={'TM': 'timestamp'}, inplace=True)
weather_2020 = pd.read_csv("weather_average2020.csv")
weather_2020.rename(columns={'TM': 'timestamp'}, inplace=True)

In [129]:
weather_2020

,timestamp,Unnamed: 0,WS_AVG,TA_AVG,TA_MAX,TA_MIN,HM_AVG,SS_DAY,RN_DAY,TA_DIF
0,2020-01-01,3.0,2.271429,-1.471429,3.228571,-6.942857,63.914286,5.328571,0.2,10.171429
1,2020-01-02,10.5,2.037500,2.550000,6.175000,-0.937500,66.725000,4.100000,0.0,7.112500
2,2020-01-03,18.5,1.337500,1.187500,7.400000,-3.075000,68.700000,7.750000,0.0,10.475000
3,2020-01-04,26.5,1.275000,0.925000,7.687500,-4.025000,75.500000,6.687500,1.5,11.712500
4,2020-01-05,34.5,0.787500,-12.062500,8.862500,-6.437500,55.475000,6.912500,0.0,15.300000
...,...,...,...,...,...,...,...,...,...,...
361,2020-12-27,2860.5,1.575000,4.850000,8.587500,0.825000,74.887500,2.025000,22.2,7.762500
362,2020-12-28,2868.5,0.750000,4.225000,12.450000,-0.837500,78.012500,7.375000,0.3,13.287500
363,2020-12-29,2876.5,2.062500,1.837500,7.300000,-1.925000,84.987500,1.500000,18.5,9.225000
364,2020-12-30,2884.5,3.937500,-7.537500,-1.875000,-10.437500,59.587500,6.012500,2.4,8.562500


In [131]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
weather_2019['timestamp'] = pd.to_datetime(weather_2019['timestamp'])
weather_2020['timestamp'] = pd.to_datetime(weather_2020['timestamp'])
weather = pd.concat([weather_2019, weather_2020])

df_concat = pd.merge(train_df, weather, on='timestamp', how='left')
df_concat.dropna(inplace=True)
df_concat

,ID,timestamp,supply(kg),price(원/kg),Unnamed: 0,WS_AVG,TA_AVG,TA_MAX,TA_MIN,HM_AVG,SS_DAY,RN_DAY,TA_DIF
0,TG_A_J_20190101,2019-01-01,0.0,0.0,3.0,2.957143,-3.042857e+00,1.771429,-7.242857,59.214286,5.571429,0.0,9.014286
1,TG_A_J_20190102,2019-01-02,0.0,0.0,10.5,2.800000,-1.437500e+00,3.087500,-5.050000,48.262500,6.487500,0.0,8.137500
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,18.5,1.312500,-3.150000e+00,4.800000,-9.500000,51.787500,8.650000,0.0,14.300000
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,26.5,1.275000,-1.300000e+00,4.387500,-7.900000,63.750000,3.775000,0.0,12.287500
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,34.5,1.750000,-2.775558e-17,5.762500,-5.562500,47.437500,7.037500,0.0,11.325000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58600,RD_F_J_20201227,2020-12-27,0.0,0.0,2860.5,1.575000,4.850000e+00,8.587500,0.825000,74.887500,2.025000,22.2,7.762500
58601,RD_F_J_20201228,2020-12-28,422000.0,606.0,2868.5,0.750000,4.225000e+00,12.450000,-0.837500,78.012500,7.375000,0.3,13.287500
58602,RD_F_J_20201229,2020-12-29,260520.0,648.0,2876.5,2.062500,1.837500e+00,7.300000,-1.925000,84.987500,1.500000,18.5,9.225000
58603,RD_F_J_20201230,2020-12-30,199620.0,686.0,2884.5,3.937500,-7.537500e+00,-1.875000,-10.437500,59.587500,6.012500,2.4,8.562500


In [132]:
df_concat['item_id'] = df_concat.ID.str[0:6]
df_concat

,ID,timestamp,supply(kg),price(원/kg),Unnamed: 0,WS_AVG,TA_AVG,TA_MAX,TA_MIN,HM_AVG,SS_DAY,RN_DAY,TA_DIF,item_id
0,TG_A_J_20190101,2019-01-01,0.0,0.0,3.0,2.957143,-3.042857e+00,1.771429,-7.242857,59.214286,5.571429,0.0,9.014286,TG_A_J
1,TG_A_J_20190102,2019-01-02,0.0,0.0,10.5,2.800000,-1.437500e+00,3.087500,-5.050000,48.262500,6.487500,0.0,8.137500,TG_A_J
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,18.5,1.312500,-3.150000e+00,4.800000,-9.500000,51.787500,8.650000,0.0,14.300000,TG_A_J
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,26.5,1.275000,-1.300000e+00,4.387500,-7.900000,63.750000,3.775000,0.0,12.287500,TG_A_J
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,34.5,1.750000,-2.775558e-17,5.762500,-5.562500,47.437500,7.037500,0.0,11.325000,TG_A_J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58600,RD_F_J_20201227,2020-12-27,0.0,0.0,2860.5,1.575000,4.850000e+00,8.587500,0.825000,74.887500,2.025000,22.2,7.762500,RD_F_J
58601,RD_F_J_20201228,2020-12-28,422000.0,606.0,2868.5,0.750000,4.225000e+00,12.450000,-0.837500,78.012500,7.375000,0.3,13.287500,RD_F_J
58602,RD_F_J_20201229,2020-12-29,260520.0,648.0,2876.5,2.062500,1.837500e+00,7.300000,-1.925000,84.987500,1.500000,18.5,9.225000,RD_F_J
58603,RD_F_J_20201230,2020-12-30,199620.0,686.0,2884.5,3.937500,-7.537500e+00,-1.875000,-10.437500,59.587500,6.012500,2.4,8.562500,RD_F_J


In [160]:
data = TimeSeriesDataFrame(df_concat.drop(columns=['ID']))
predictor = TimeSeriesPredictor( 
    prediction_length=1,
    target="price(원/kg)",
    eval_metric="RMSE",
)
# seed 고정
predictor.fit( data, random_seed=42, )

Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels/ag-20241026_011731'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #47~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Oct  2 16:16:55 UTC 2
CPU Count:          16
GPU Count:          1
Memory Avail:       8.89 GB / 15.25 GB (58.3%)
Disk Space Avail:   397.79 GB / 467.89 GB (85.0%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': RMSE,
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 42,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'price(원/kg)',
 'verbosity': 2}



Inferred time series frequency: 'D'
Provided train_data has 28509 rows, 39 time series. Median time series length is 731 (min=731, max=731). 

Provided data contains following columns:
	target: 'price(원/kg)'
	past_covariates:
		categorical:        []
		continuous (float): ['supply(kg)', 'Unnamed: 0', 'WS_AVG', 'TA_AVG', 'TA_MAX', 'TA_MIN', ...]

To learn how to fix incorrectly inferred types, please see documentation for TimeSeriesPredictor.fit

AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2024-10-26 10:17:31
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'CrostonSBA', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'AutoARIMA', 'Chronos[base]', 'TemporalFusionTransformer', 'DeepAR', 'PatchTST']
Training timeseries model SeasonalNaive. 
	-649.6149     

In [161]:
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	17.56   s     = Training runtime
Fitting model: DirectTabular_FULL
	0.52    s     = Training runtime
Fitting model: CrostonSBA_FULL | Skipping fit via cloning parent ...
Fitting model: NPTS_FULL | Skipping fit via cloning parent ...
Fitting model: DynamicOptimizedTheta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: AutoARIMA_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['SeasonalNaive_FULL', 'RecursiveTabular_FULL', 'DirectTabu

{'SeasonalNaive': 'SeasonalNaive_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DirectTabular': 'DirectTabular_FULL',
 'CrostonSBA': 'CrostonSBA_FULL',
 'NPTS': 'NPTS_FULL',
 'DynamicOptimizedTheta': 'DynamicOptimizedTheta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'AutoARIMA': 'AutoARIMA_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [166]:
pred = predictor.predict(data, random_seed=42, )
pred = pred.reset_index()

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [174]:
def price_comparison(pred, df_concat, date):
    date = pd.to_datetime(date)
    pred = pred[['item_id', "timestamp",'mean']]
    df_previous = df_concat[df_concat['timestamp'] == date - pd.DateOffset(years=1)].copy()
    df_previous['item_id'] = df_previous.ID.str[0:6]

    comparison_df = pd.merge(
    df_previous[['item_id', 'price(원/kg)']].rename(columns={'price(원/kg)': 'price_previous'}),
    pred[['item_id', 'mean']].rename(columns={'mean': 'mean_current'}),
    on='item_id',
    how='inner'
                )
    comparison_df['price_increase_percent'] = ((comparison_df['mean_current'] - comparison_df['price_previous']) / comparison_df['price_previous']) * 100
    comparison_df[['item_id', 'price_previous', 'mean_current', 'price_increase_percent']]
    comparison_df = comparison_df[np.isfinite(comparison_df['price_increase_percent'])] # 값 폭주하는 거 제거
    comparison_df = comparison_df[comparison_df['price_increase_percent'] > 0] # 양수만 남김
    item_mapping = {
        'TG': '감귤',
        'BC': '브로콜리',
        'RD': '무',
        'CR': '당근',
        'CB': '양배추'
    }
    
    # item_id의 첫 두 글자를 통해 품목 이름을 매핑하여 새로운 열 추가
    comparison_df['item_name'] = comparison_df['item_id'].str[:2].map(item_mapping)
    
    # 열 이름 변경
    comparison_df.rename(columns={'mean': 'mean_current'}, inplace=True)
    
    # 결과 반환
    return comparison_df[['item_id', 'item_name', 'price_previous', 'mean_current', 'price_increase_percent']]


    return comparison_df


In [175]:
compare = price_comparison(pred, df_concat, date = "2020-01-03")
compare

,item_id,item_name,price_previous,mean_current,price_increase_percent
4,TG_C_J,감귤,1965.0,2034.278809,3.525639
37,CB_F_J,양배추,478.0,556.227600,16.365607
38,RD_F_J,무,395.0,487.160706,23.331824


In [176]:
compare.to_json("comparison_results.json", orient="records", force_ascii=False)

### mean이 예측값입니다

X년 Y월 Z일이 들어오면 X-1년 Y월 Z일의 가격 값이랑 비교. -> 상승 폭 제시

In [152]:
# df_2019 = df_concat[df_concat['timestamp'] == '2019-01-03'].copy()
# df_2020 = pred[pred['timestamp'] == '2020-01-01'].copy()
# df_2020['item_id'] = df_concat.ID.str[0:6]

In [153]:
# # Step 3: Merge and rename columns
# comparison_df = pd.merge(
#     df_2019[['item_id', 'price(원/kg)']].rename(columns={'price(원/kg)': 'price_2019'}),
#     df_2020[['item_id', 'mean']].rename(columns={'mean': 'mean_2020'}),
#     on='item_id',
#     how='inner'
# )

# # Step 4: Calculate the percentage increase from 2019 to 2020
# comparison_df['price_increase_percent'] = ((comparison_df['mean_2020'] - comparison_df['price_2019']) / comparison_df['price_2019']) * 100

# # Display the result
# comparison_df[['item_id', 'price_2019', 'mean_2020', 'price_increase_percent']]